In [2]:
!CXXFLAGS+=-stdlib=libc++ maturin develop --release

🔗 Found pyo3 bindings
🐍 Found CPython 3.11 at /Users/josh/josh/projects/tech/riversim/.venv/bin/python
💻 Using `MACOSX_DEPLOYMENT_TARGET=10.7` for x86_64-apple-darwin by default
   Compiling riversim v0.1.0 (/Users/josh/josh/projects/tech/riversim)
    Finished release [optimized + debuginfo] target(s) in 11.70s              
📦 Built wheel for CPython 3.11 to /var/folders/w9/c8qym84j2pg4fjvx_bzr84bc0000gn/T/.tmp0APBzp/riversim-0.1.0-cp311-cp311-macosx_10_7_x86_64.whl
🛠 Installed riversim-0.1.0


In [1]:
import contextplot
import matplotlib
import matplotlib.collections
import matplotlib.patches
import numpy as np
import tqdm

import riversim

contextplot.set_defaults()

example = 'bump_1d'
# example = 'singularity_1d'

num_cells = 60
solver = getattr(riversim, example)(num_cells, 20)

dt = 0.002

with contextplot.context_video(f'{example}.mp4', 1, size_inches=(8, 2.5), frame_rate_hz=20) as cv:
    bar = tqdm.tqdm(range(1000))
    for step in bar:
        if step % 10 == 0:
            with cv.next_frame() as cp:
                z_lattice = solver.z_lattice
                x_vertices = solver.grid.x_axis.vertices
                x_centers = solver.grid.x_axis.centers
                pressure = solver.pressure
                velocity = solver.velocity

                poly_verts = []
                pressures = []
                for i in range(z_lattice.shape[0] - 1):
                    for j in range(z_lattice.shape[2] - 1):
                        poly_verts.append([
                            [x_vertices[i], z_lattice[i, 0, j]],
                            [x_vertices[i + 1], z_lattice[i + 1, 0, j]],
                            [x_vertices[i + 1], z_lattice[i + 1, 0, j + 1]],
                            [x_vertices[i], z_lattice[i, 0, j + 1]],
                        ])
                        pressures.append(pressure[i, 0, j, 1])
                poly_collection = matplotlib.collections.PolyCollection(
                    poly_verts, closed=True, array=pressures, cmap=matplotlib.cm.viridis, edgecolors='black', linewidth=0.2, snap=True)
                outs = cp.ax_raw.add_collection(poly_collection)
                _ = cp.figure.colorbar(outs)

                cp.ax.quiver(
                    np.broadcast_to(x_centers[:, np.newaxis], z_lattice[:-1, 0, :-1].shape),
                    0.25 * (z_lattice[:-1, 0, :-1] + z_lattice[1:, 0, :-1] + z_lattice[:-1, 0, 1:] + z_lattice[1:, 0, 1:]),
                    velocity[:, 0, :, 0, 0],
                    velocity[:, 0, :, 0, 2],
                    pivot='tail',
                    scale=1,
                    scale_units='inches',
                )

                cp.ax.set(xlim=[solver.grid.x_axis.vertices[0], solver.grid.x_axis.vertices[-1]], ylim=[0, 1.5], aspect='equal')
                _ = bar.set_description(f't: {step * dt:.4f}, Courant margin: {solver.courant_dt / dt:.4f}', 50)

        try:
            solver.step(dt)
        except:
            break

t: 1.7800, Courant margin: 4.4910:  90%|████████▉ | 896/1000 [01:07<00:07, 13.26it/s]stack backtrace:
   0: rust_begin_unwind
             at /rustc/90c541806f23a127002de5b4038be731ba1458ca/library/std/src/panicking.rs:578:5
   1: core::panicking::panic_fmt
             at /rustc/90c541806f23a127002de5b4038be731ba1458ca/library/core/src/panicking.rs:67:14
   2: core::result::unwrap_failed
             at /rustc/90c541806f23a127002de5b4038be731ba1458ca/library/core/src/result.rs:1687:5
   3: core::result::Result<T,E>::unwrap
             at /rustc/90c541806f23a127002de5b4038be731ba1458ca/library/core/src/result.rs:1089:23
   4: riversim::physics::Solver::compute_pressure
             at ./src/physics.rs:242:9
   5: riversim::physics::Solver::step
             at ./src/physics.rs:197:32
   6: riversim::physics::Solver::step_py
             at ./src/physics.rs:285:9
   7: riversim::physics::_::<impl riversim::physics::Solver>::__pymethod_step__
             at ./src/physics.rs:252:1
   8:

In [2]:
(pressure[5, 0, 1, 1] - pressure[5, 0, 0, 1]) / (z_lattice[5, 0, 1] - z_lattice[5, 0, 0])

-9755.998464128003